<font color="red">This has **not** been revised (raw + not edited)! There may be material in here to help you integrate Jupyter, Elixir, and libraries but that's all. The narrative here isn't cohesive. Our integration w/ Elixir's Stream is terrible + should not be done this way but you might find parts of it instructive.</font>

# Class #4

We were going to integrate our generators with Elixir's Stream and StreamData

---

## Using Caffeine

This is the only way I've found to use a library that's not in the Elixir core:
we navigate to the directory and then load the module.

In [1]:
File.cwd!() =~ "project/caffeine"

false

In [2]:
cd("~/project/caffeine")

/Users/josephyiasemides/project/caffeine


In [3]:
File.cwd!() =~ "project/caffeine"

true

In [4]:
Code.load_file("lib/caffeine.ex")

[{Caffeine.Element, <<70, 79, 82, 49, 0, 0, 3, 208, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 130, 0, 0, 0, 12, 23, 69, 108, 105, 120, 105, 114, 46, 67, 97, 102, 102, 101, 105, 110, 101, 46, 69, 108, 101, 109, 101, 110, ...>>}, {Caffeine.Stream, <<70, 79, 82, 49, 0, 0, 19, 88, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 1, 24, 0, 0, 0, 32, 22, 69, 108, 105, 120, 105, 114, 46, 67, 97, 102, 102, 101, 105, 110, 101, 46, 83, 116, 114, 101, 97, ...>>}, {Caffeine, <<70, 79, 82, 49, 0, 0, 3, 220, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 122, 0, 0, 0, 12, 15, 69, 108, 105, 120, 105, 114, 46, 67, 97, 102, 102, 101, 105, 110, 101, 8, 95, 95, 105, 110, ...>>}]

Once we have Caffeine ready to use we change our definition of the LCG from last time to use Caffeine instead of the improvised stream we'd built (see the `stream/2` function). The `LCG.MMIX` module and the expressions imediately following are borrowed from last time.

In [5]:
defmodule LCG do
  defmodule Parameters do
    defstruct [
      modulus:    9,
      multiplier: 2,
      increment:  0
    ]

    def okay?(p) do
      m = p.modulus
      a = p.multiplier
      c = p.increment
      
      x = 0 < m
      y = (0 < a and a < m)
      z = (0 <= c and c < m)

      x and y and z
    end
  end

  def instance(p) do
    m = p.modulus; a = p.multiplier; c = p.increment
    fn s when 0 <= s and s < m ->
      Integer.mod(a * s + c, m)
    end
  end

  def stream(seed, generator) do
    f = fn ->
      stream(generator.(seed), generator)
    end
    Caffeine.Stream.construct(seed, f)
  end
end

{:module, LCG, <<70, 79, 82, 49, 0, 0, 8, 56, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 1, 27, 0, 0, 0, 28, 10, 69, 108, 105, 120, 105, 114, 46, 76, 67, 71, 8, 95, 95, 105, 110, 102, 111, 95, 95, 9, 102, 117, ...>>, {:stream, 2}}

In [6]:
defmodule LCG.MMIX do
  def instance do
    x = %LCG.Parameters{
      modulus:    round(:math.pow(2, 64)),
      multiplier: 6364136223846793005,
      increment:  1442695040888963407
    }
    true = LCG.Parameters.okay?(x)
    LCG.instance(x)
  end
end

{:module, LCG.MMIX, <<70, 79, 82, 49, 0, 0, 5, 160, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 175, 0, 0, 0, 17, 15, 69, 108, 105, 120, 105, 114, 46, 76, 67, 71, 46, 77, 77, 73, 88, 8, 95, 95, 105, 110, 102, 111, ...>>, {:instance, 0}}

In [7]:
m = LCG.MMIX.instance()

#Function<0.36907607/1 in LCG.instance/1>

In [8]:
Caffeine.Stream.take(LCG.stream(1, m), 9)

[1, 7806831264735756412, 9396908728118811419, 11960119808228829710, 7062582979898595269, 14673421054488193520, 9232803539723513983, 10218303843513747618, 1206773305466921929]

The LCG stream imposes on us large natural numbers. We have to build a way to work with them so that we can build data generators that are reasonable. I.e. we don't want to generate lists that are 7806831264735756412 elements long.

In [9]:
defmodule Generator.BoundNatural do
  def stream(limit: i) do
    s = LCG.stream(1, LCG.MMIX.instance())
    Caffeine.Stream.map(s, limit(i))
  end

  defp limit(i) do
    fn x ->
      Integer.mod(x, i)
    end
  end
end

{:module, Generator.BoundNatural, <<70, 79, 82, 49, 0, 0, 6, 36, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 247, 0, 0, 0, 22, 29, 69, 108, 105, 120, 105, 114, 46, 71, 101, 110, 101, 114, 97, 116, 111, 114, 46, 66, 111, 117, 110, 100, ...>>, {:limit, 1}}

In [10]:
x = Caffeine.Stream.take(Generator.BoundNatural.stream(limit: 100), 100)

[1, 12, 19, 10, 69, 20, 83, 18, 29, 44, 31, 54, 93, 48, 55, 58, 17, 52, 83, 46, 25, 24, 39, 46, 21, 48, 35, 86, 25, 48, 39, 38, 29, 72, 79, 38, 89, 28, 19, 78, 33, 48, 27, 22, 37, 76, 83, 98, 5, 28, ...]

Let's test that our distribution of _small_ natural numbers is uniform.

In [11]:
Enum.count(x, fn x -> x >= 50 end)

47

There's more than one way to write a stream of cube numbers (and other streams):
1. `Cube.V1` uses the primitives in Caffeine (like `construct/2`) and increments a counter
2. `Cube.V2` does away with the counter by `map`ing over the `Natural` stream

In [12]:
defmodule Cube.V1 do
  def stream do
    stream(0)
  end

  defp stream(x) do
    f = fn ->
      stream(x + 1)
    end
    Caffeine.Stream.construct(cube(x), f)
  end

  defp cube(x) do
    x * x * x
  end
end

{:module, Cube.V1, <<70, 79, 82, 49, 0, 0, 5, 204, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 187, 0, 0, 0, 19, 14, 69, 108, 105, 120, 105, 114, 46, 67, 117, 98, 101, 46, 86, 49, 8, 95, 95, 105, 110, 102, 111, 95, ...>>, {:cube, 1}}

In [13]:
Caffeine.Stream.take(Cube.V1.stream(), 7)

[0, 1, 8, 27, 64, 125, 216]

In [14]:
defmodule Natural do
  def stream do
    stream(0)
  end

  defp stream(x) do
    f = fn ->
      stream(x + 1)
    end
    Caffeine.Stream.construct(x, f)
  end
end

{:module, Natural, <<70, 79, 82, 49, 0, 0, 5, 84, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 180, 0, 0, 0, 17, 14, 69, 108, 105, 120, 105, 114, 46, 78, 97, 116, 117, 114, 97, 108, 8, 95, 95, 105, 110, 102, 111, 95, ...>>, {:stream, 1}}

In [15]:
Caffeine.Stream.take(Natural.stream(), 7)

[0, 1, 2, 3, 4, 5, 6]

In [16]:
defmodule Cube.V2 do
  def stream do
    Caffeine.Stream.map(Natural.stream(), &cube/1)
  end

  defp cube(x) do
    x * x * x
  end
end

{:module, Cube.V2, <<70, 79, 82, 49, 0, 0, 5, 100, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 194, 0, 0, 0, 19, 14, 69, 108, 105, 120, 105, 114, 46, 67, 117, 98, 101, 46, 86, 50, 8, 95, 95, 105, 110, 102, 111, 95, ...>>, {:cube, 1}}

In [17]:
Caffeine.Stream.take(Cube.V2.stream(), 5)

[0, 1, 8, 27, 64]

This is **one** way to write a generator for printable ASCII characters.

In [18]:
defmodule Generator.Character.PrintableASCII do
  def stream do
    f = fn (x) ->
      lower_bound() + x
    end
    Caffeine.Stream.map(Generator.BoundNatural.stream(limit: length()), f)
  end

  defp length do
    upper_bound - lower_bound
  end

  defp lower_bound do
    ?\s
  end

  defp upper_bound do
    ?~
  end
end

{:module, Generator.Character.PrintableASCII, <<70, 79, 82, 49, 0, 0, 6, 164, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 1, 14, 0, 0, 0, 23, 41, 69, 108, 105, 120, 105, 114, 46, 71, 101, 110, 101, 114, 97, 116, 111, 114, 46, 67, 104, 97, 114, 97, ...>>, {:upper_bound, 0}}

In [19]:
Caffeine.Stream.take(Generator.Character.PrintableASCII.stream(), 96)

'!he@E@?l{$w@7$O:!2[:-LIBG@[&!^7:gp]f% W4S8qjgve,s@#$}vul?B;(GLGZ_J9Ha>!B%8#T]6q^iD[hY"kJM\\-8u4?,'

**One** way to write a generator for (ASCII) character lists.

In [20]:
defmodule Generator.CharacterList do
  alias Generator.BoundNatural
  alias Generator.Character.PrintableASCII

  def stream do
    stream(length: BoundNatural.stream(limit: 100), fill: PrintableASCII.stream())
  end

  defp stream(length: s, fill: r) do
    {x, y} = split(r, Caffeine.Stream.head(s))
    f = fn ->
      stream(length: Caffeine.Stream.tail(s), fill: y)
    end
    Caffeine.Stream.construct(x, f)
  end

  defp split(s, i) do
    {Caffeine.Stream.take(s, i), rest(s, i)}
  end

  defp rest(s, 0) do
    s
  end
  defp rest(s, i) when i > 0 do
    rest(Caffeine.Stream.tail(s), i - 1)
  end
end

{:module, Generator.CharacterList, <<70, 79, 82, 49, 0, 0, 8, 124, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 1, 50, 0, 0, 0, 26, 30, 69, 108, 105, 120, 105, 114, 46, 71, 101, 110, 101, 114, 97, 116, 111, 114, 46, 67, 104, 97, 114, 97, ...>>, {:rest, 2}}

In [21]:
x = Caffeine.Stream.take(Generator.CharacterList.stream(), 10)

['!', 'he@E@?l{$w@7', '$O:!2[:-LIBG@[&!^7:', 'gp]f% W4S8', 'qjgve,s@#$}vul?B;(GLGZ_J9Ha>!B%8#T]6q^iD[hY"kJM\\-8u4?,-J_:WV/L=Z]Pi$}', 'deD-.a>wR-@u\\O>/0c&I', 'tkh!h=rg2/N9r{81&}"74=^IVK>#<;<mBar#:CnC0m\\?6GLeXs$%:c\\_XWrs^/,[0i|SJa<q6)|= c&sp) ', 'G2aJ?*#RsPS"c0\'RGR', 'KVSN\'2i4!$SX;*7Re(/T{F-@]*ono', '`A&#$K8Q$kri\\Gp!hKHWZC6g6qb1v-L\'j_.?v%XCLE*A']

## Summary

Some of the things we've seen:

- Building streams with primitives (e.g. `Caffeine.Stream.{sentinel/0, construct/2}`
- Building streams with HOFs (e.g. `Caffeine.Stream.{map/2, filter/2}`)
- [Keyword lists](https://elixir-lang.org/getting-started/keywords-and-maps.html#keyword-lists):
The `Rectangle` example in [Smalltalk's Wikipedia article](https://en.wikipedia.org/wiki/Smalltalk#Messages) motivates their use for clarity at call site.

## Data Generators

- Natural

- Character

- Integer

- Float

- Boolean and `nil`

- List:

  A homogeneously typed list (elements are of the same type).
  API suggestion:
  `List.stream(g)` where `g` is any other generator.
  E.g. `List.stream(Integer.stream())` would build elements like `[1,8,0]`.


- Character List

- Pair (i.e. `[_|?]` where `?` isn't `[]`)

- Binary

- String

- Tuple:

  A heterogeneously typed tuple.
  API suggestion:
  `Tuple.stream(l)` where `l` is a list of generators.
  E.g. if `l = [Integer.stream(), Float.stream(), String.stream()]` then the elements would look like `{5, 6.1, "qwerty"}`.


- Record

- Symbol/Atom

- Map/Struct

- Function:

  This could be as simple as `Function.stream(arity: 0, return: Integer.stream())` with no restriction on the input parameters.
  Let's restrict arity to 5.
  E.g.:
  ```Elixir
  > f = head(Function.stream(arity: 1, return: String.stream()));
  > f.(nil)
  > "qwerty"
  ```
  
- Term:

  Every element produced by this generator is a value from a randomly selected generator.


- Port, PID, and Ref


In [22]:
try do
  Enum.take(Generator.Character.PrintableASCII.stream(), 10) && "Yey!"
rescue
  FunctionClauseError ->
    "Ney!"
end

"Ney!"

In [23]:
defmodule Generator.Character.PrintableASCII do
  defstruct [
     :stream
  ]

  def stream do
    f = fn (x) ->
      lower_bound() + x
    end
    Caffeine.Stream.map(Generator.BoundNatural.stream(limit: length()), f)
  end

  defp length do
    upper_bound - lower_bound
  end

  defp lower_bound do
    ?\s
  end

  defp upper_bound do
    ?~
  end
end

{:module, Generator.Character.PrintableASCII, <<70, 79, 82, 49, 0, 0, 8, 176, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 1, 77, 0, 0, 0, 29, 41, 69, 108, 105, 120, 105, 114, 46, 71, 101, 110, 101, 114, 97, 116, 111, 114, 46, 67, 104, 97, 114, 97, ...>>, {:upper_bound, 0}}

In [24]:
try do
  Enum.take(Generator.Character.PrintableASCII.stream(), 10) && "Yey!"
rescue
  FunctionClauseError ->
    "Ney!"
end

"Ney!"

In [25]:
defimpl Enumerable, for: Generator.Character.PrintableASCII do
  def reduce(_, {:halt, acc}, _) do
    {:halt, acc}
  end
  def reduce(s, {:suspend, acc}, f) do
    {:suspended, acc, &reduce(s.stream, &1, f)}
  end
  def reduce(s, {:cont, acc}, f) do
    cond do
      Caffeine.Stream.sentinel?(s.stream) ->
        {:done, acc}
      Caffeine.Stream.construct?(s.stream) ->
        reduce(%Generator.Character.PrintableASCII{stream: Caffeine.Stream.tail(s.stream)}, f.(Caffeine.Stream.head(s.stream), acc), f)
    end
  end

  def count(_) do
    {:error, Generator.Character.PrintableASCII}
  end

  def member?(_, _) do
    {:error, Generator.Character.PrintableASCII}
  end

  def slice(_) do
    {:error, Generator.Character.PrintableASCII}
  end
end

{:module, Enumerable.Generator.Character.PrintableASCII, <<70, 79, 82, 49, 0, 0, 12, 20, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 1, 176, 0, 0, 0, 40, 52, 69, 108, 105, 120, 105, 114, 46, 69, 110, 117, 109, 101, 114, 97, 98, 108, 101, 46, 71, 101, 110, 101, ...>>, {:__impl__, 1}}

In [26]:
Enum.take(%Generator.Character.PrintableASCII{stream: Generator.Character.PrintableASCII.stream()}, 10)

'!he@E@?l{$'

In [27]:
File.ls!("deps/stream_data/lib/")

["ex_unit_properties.ex", "stream_data.ex", "stream_data"]

In [28]:
File.ls!("deps/stream_data/lib/stream_data")

["lazy_tree.ex"]

In [29]:
x = ["deps/stream_data/lib/stream_data/lazy_tree.ex", "deps/stream_data/lib/stream_data.ex", "deps/stream_data/lib/ex_unit_properties.ex"]

["deps/stream_data/lib/stream_data/lazy_tree.ex", "deps/stream_data/lib/stream_data.ex", "deps/stream_data/lib/ex_unit_properties.ex"]

In [30]:
Enum.map(x, &Code.load_file/1)

[[{Inspect.StreamData.LazyTree, <<70, 79, 82, 49, 0, 0, 10, 16, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 1, 101, 0, 0, 0, 34, 34, 69, 108, 105, 120, 105, 114, 46, 73, 110, 115, 112, 101, 99, 116, 46, 83, 116, 114, 101, 97, 109, ...>>}, {StreamData.LazyTree, <<70, 79, 82, 49, 0, 0, 24, 80, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 2, 8, 0, 0, 0, 48, 26, 69, 108, 105, 120, 105, 114, 46, 83, 116, 114, 101, 97, 109, 68, 97, 116, 97, 46, 76, 97, ...>>}], [{StreamData.FilterTooNarrowError, <<70, 79, 82, 49, 0, 0, 22, 44, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 1, 195, 0, 0, 0, 41, 38, 69, 108, 105, 120, 105, 114, 46, 83, 116, 114, 101, 97, 109, 68, 97, 116, 97, 46, 70, 105, ...>>}, {StreamData.TooManyDuplicatesError, <<70, 79, 82, 49, 0, 0, 16, 140, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 1, 172, 0, 0, 0, 38, 40, 69, 108, 105, 120, 105, 114, 46, 83, 116, 114, 101, 97, 109, 68, 97, 116, 97, 46, 84, ...>>}, {Enumerable.StreamData, <<70, 79, 82, 49, 0, 0, 7, 228, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0,

In [31]:
ExUnit.start()

[#Function<2.80697779/1 in ExUnit.start/1>]

In [32]:
defmodule Generator.Character.PrintableASCIITest do
  use ExUnit.Case
  use ExUnitProperties

  property "fu" do
    generator = %Generator.Character.PrintableASCII{stream: Generator.Character.PrintableASCII.stream()}
    check all e <- generator do
      IO.inspect(e)
    end
  end
end

{:module, Generator.Character.PrintableASCIITest, <<70, 79, 82, 49, 0, 0, 18, 36, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 3, 8, 0, 0, 0, 62, 45, 69, 108, 105, 120, 105, 114, 46, 71, 101, 110, 101, 114, 97, 116, 111, 114, 46, 67, 104, 97, 114, 97, ...>>, {:"property fu", 1}}

In [33]:
Generator.Character.PrintableASCIITest.__info__(:functions)

[__ex_unit__: 0, __ex_unit__: 2, "property fu": 1]

In [34]:
Generator.Character.PrintableASCIITest."property fu"(%{})

ArgumentError: 1

In [34]:
Code.load_file("deps/stream_data/mix.exs")

[{StreamData.Mixfile, <<70, 79, 82, 49, 0, 0, 8, 148, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 197, 0, 0, 0, 20, 25, 69, 108, 105, 120, 105, 114, 46, 83, 116, 114, 101, 97, 109, 68, 97, 116, 97, 46, 77, 105, 120, 102, ...>>}]

In [35]:
:observer.start()

:ok

In [36]:
Application.get_all_env(:ex_unit)

[stacktrace_depth: 20, colors: [], refute_receive_timeout: 100, formatters: [ExUnit.CLIFormatter], slowest: 0, module_load_timeout: 60000, autorun: false, include: [], plural_rules: %{"property" => "properties"}, trace: false, capture_log: false, assert_receive_timeout: 100, exclude: [], included_applications: [], timeout: 60000]

In [37]:
Application.get_all_env(:stream_data)

[]

In [38]:
configuration = [
        initial_size: 1,
        max_runs: 100,
        max_run_time: :infinity,
        max_shrinking_steps: 100
      ]

[initial_size: 1, max_runs: 100, max_run_time: :infinity, max_shrinking_steps: 100]

In [39]:
f = fn {name, value} ->
  Application.put_env(:stream_data, name, value)
end
Enum.each(configuration, f)

:ok

In [40]:
x = ExUnit.configuration(); x[:seed]

685408

In [41]:
generator = %Generator.Character.PrintableASCII{stream: Generator.Character.PrintableASCII.stream()}

%Generator.Character.PrintableASCII{stream: [33 | #Function<1.9917893/0 in Caffeine.Stream.map/2>]}

In [42]:
Enum.take generator, 5

'!he@E'

In [43]:
3 + 2

5

In [44]:
defmodule M do
  def f(fu: x, bar: y) do
    x * y
  end
end

{:module, M, <<70, 79, 82, 49, 0, 0, 4, 104, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 126, 0, 0, 0, 16, 8, 69, 108, 105, 120, 105, 114, 46, 77, 8, 95, 95, 105, 110, 102, 111, 95, 95, 9, 102, 117, 110, 99, ...>>, {:f, 1}}

In [45]:
z = [fu: x, bar: y]

CompileError: 1

In [45]:
defmodule M do
  def f([fu: x, bar: y]) do
    x * y
  end
end

{:module, M, <<70, 79, 82, 49, 0, 0, 4, 104, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 126, 0, 0, 0, 16, 8, 69, 108, 105, 120, 105, 114, 46, 77, 8, 95, 95, 105, 110, 102, 111, 95, 95, 9, 102, 117, 110, 99, ...>>, {:f, 1}}

In [46]:
defmodule M do
  def f([{:fu, x}, {:bar, y}]) do
    x * y
  end
end

{:module, M, <<70, 79, 82, 49, 0, 0, 4, 104, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 126, 0, 0, 0, 16, 8, 69, 108, 105, 120, 105, 114, 46, 77, 8, 95, 95, 105, 110, 102, 111, 95, 95, 9, 102, 117, 110, 99, ...>>, {:f, 1}}

In [47]:
defmodule M do
  def f([fu: x, bar: y] \\ [fu: 5, bar: 3]) do
    x * y
  end
end

{:module, M, <<70, 79, 82, 49, 0, 0, 4, 216, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 126, 0, 0, 0, 16, 8, 69, 108, 105, 120, 105, 114, 46, 77, 8, 95, 95, 105, 110, 102, 111, 95, 95, 9, 102, 117, 110, 99, ...>>, {:f, 1}}

In [48]:
M.f()

15

In [49]:
M.f(fu: 1, bar: 1)

1

In [50]:
defmodule Generator.Character.PrintableASCII do
  def stream do
    Caffeine.Stream.map(Generator.BoundNatural.stream(limit: length()), &value/1)
  end

  def value(x) when is_integer(x) do
    lower_bound() + x
  end

  defp length do
    upper_bound - lower_bound
  end

  defp lower_bound do
    ?\s
  end

  defp upper_bound do
    ?~
  end
end

{:module, Generator.Character.PrintableASCII, <<70, 79, 82, 49, 0, 0, 6, 236, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 1, 20, 0, 0, 0, 24, 41, 69, 108, 105, 120, 105, 114, 46, 71, 101, 110, 101, 114, 97, 116, 111, 114, 46, 67, 104, 97, 114, 97, ...>>, {:upper_bound, 0}}

In [51]:
Caffeine.Stream.take(Generator.Character.PrintableASCII.stream(), 5)

'!he@E'

In [52]:
[Generator.Character.PrintableASCII.value(5)]

'%'

In [53]:
[Generator.Character.PrintableASCII.value(0)]

' '

In [54]:
[Generator.Character.PrintableASCII.value(255)]

[287]

In [55]:
defmodule M do
  def f(arity: 0, return: r) do
  end
  def f(arity: 1, return: r) do
  end
  def f(arity: 2, return: r) do
  end
end

{:module, M, <<70, 79, 82, 49, 0, 0, 4, 100, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 134, 0, 0, 0, 16, 8, 69, 108, 105, 120, 105, 114, 46, 77, 8, 95, 95, 105, 110, 102, 111, 95, 95, 9, 102, 117, 110, 99, ...>>, {:f, 1}}

In [56]:
M.f(arity: 5, return: nil)

FunctionClauseError: 1

In [56]:
## Call site reability

nil

In [57]:
defmodule M do
  def f0(return: r) do
  end

  def f1(return: r) do
  end

  def f2(return: r) do
  end
end

{:module, M, <<70, 79, 82, 49, 0, 0, 4, 172, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 135, 0, 0, 0, 17, 8, 69, 108, 105, 120, 105, 114, 46, 77, 8, 95, 95, 105, 110, 102, 111, 95, 95, 9, 102, 117, 110, 99, ...>>, {:f2, 1}}

# Good tests


* distict test case
* explicit

In [58]:
defmodule KernelTest do
  use ExUnit.Case

  test "abs/1 is always positve" do
    table = [
      {-5, +5}, ## -ve
      {0,  0},
      {+3, +3} ## +ve
    ]
    for {i, o} <- table do
      assert Kernel.abs(i) === o
    end
  end
end

{:module, KernelTest, <<70, 79, 82, 49, 0, 0, 10, 56, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 1, 65, 0, 0, 0, 30, 17, 69, 108, 105, 120, 105, 114, 46, 75, 101, 114, 110, 101, 108, 84, 101, 115, 116, 8, 95, 95, 105, 110, ...>>, {:"test abs/1 is always positve", 1}}

In [59]:
KernelTest."test abs/1 is always positve"(%{})

:ok

In [60]:
defmodule PoisonTest do
  use ExUnit.Case

  test "" do
    ## {"fu": "bar"}
    key = 4
    value = 5
    struct = 4
    assert byte(Poison.encode(%{fu: "bar"})) === key + value + struct
  end
end

CompileError: 1

In [60]:
defmodule Generator.Character.PrintableASCII do
  def stream do
    f = fn (x) ->
      lower_bound() + x
    end
    Caffeine.Stream.map(Generator.BoundNatural.stream(limit: length()), f)
  end

  defp length do
    upper_bound - lower_bound
  end

  defp lower_bound do
    ?\s
  end

  defp upper_bound do
    ?~
  end
end

{:module, Generator.Character.PrintableASCII, <<70, 79, 82, 49, 0, 0, 6, 164, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 1, 14, 0, 0, 0, 23, 41, 69, 108, 105, 120, 105, 114, 46, 71, 101, 110, 101, 114, 97, 116, 111, 114, 46, 67, 104, 97, 114, 97, ...>>, {:upper_bound, 0}}

In [61]:
Caffeine.Stream.take(Generator.Character.PrintableASCII.stream(), 96)

'!he@E@?l{$w@7$O:!2[:-LIBG@[&!^7:gp]f% W4S8qjgve,s@#$}vul?B;(GLGZ_J9Ha>!B%8#T]6q^iD[hY"kJM\\-8u4?,'

In [62]:
Enum.take(Generator.Character.PrintableASCII.stream(), 96)

FunctionClauseError: 1

In [62]:
Enum.take(1..100, 5)

[1, 2, 3, 4, 5]

In [63]:
defmodule Generator.Character.PrintableASCII do
  defstruct [
    :stream
  ]

  def stream do
    f = fn (x) ->
      lower_bound() + x
    end
    Caffeine.Stream.map(Generator.BoundNatural.stream(limit: length()), f)
  end

  defp length do
    upper_bound - lower_bound
  end

  defp lower_bound do
    ?\s
  end

  defp upper_bound do
    ?~
  end
end

{:module, Generator.Character.PrintableASCII, <<70, 79, 82, 49, 0, 0, 8, 176, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 1, 77, 0, 0, 0, 29, 41, 69, 108, 105, 120, 105, 114, 46, 71, 101, 110, 101, 114, 97, 116, 111, 114, 46, 67, 104, 97, 114, 97, ...>>, {:upper_bound, 0}}

In [64]:
defimpl Enumerable, for: Generator.Character.PrintableASCII do
end

{:module, Enumerable.Generator.Character.PrintableASCII, <<70, 79, 82, 49, 0, 0, 5, 232, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 248, 0, 0, 0, 18, 52, 69, 108, 105, 120, 105, 114, 46, 69, 110, 117, 109, 101, 114, 97, 98, 108, 101, 46, 71, 101, 110, 101, ...>>, {:__impl__, 1}}

In [65]:
%Generator.Character.PrintableASCII{}

%Generator.Character.PrintableASCII{stream: nil}

In [66]:
defimpl Enumerable, for: Generator.Character.PrintableASCII do

  def reduce(_, {:halt, acc}, _) do
    {:halted, acc}
  end
  def reduce(s, {:suspend, acc}, f) do
    {:suspend, acc, &reduce(s, &1, f)}
  end
  def reduce(s, {:cont, acc}, f) do
    import Caffeine.Stream

    s = s.stream
    cond do
      sentinel?(s) ->
        {:done, acc}
      construct?(s) ->
        reduce(tail(s), f.(head(s), acc), f)
    end
  end
end

{:module, Enumerable.Generator.Character.PrintableASCII, <<70, 79, 82, 49, 0, 0, 9, 180, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 1, 142, 0, 0, 0, 36, 52, 69, 108, 105, 120, 105, 114, 46, 69, 110, 117, 109, 101, 114, 97, 98, 108, 101, 46, 71, 101, 110, 101, ...>>, {:__impl__, 1}}

In [67]:
defimpl Enumerable, for: Generator.Character.PrintableASCII do

  def count(_) do
    {:error, Generator.Character.PrintableASCII}
  end

  def member(_) do
    {:error, Generator.Character.PrintableASCII}
  end

  def count(_) do
    {:error, Generator.Character.PrintableASCII}
  end

  def reduce(_, {:halt, acc}, _) do
    {:halted, acc}
  end
  def reduce(s, {:suspend, acc}, f) do
    {:suspend, acc, &reduce(s, &1, f)}
  end
  def reduce(s, {:cont, acc}, f) do
    import Caffeine.Stream

    s = s.stream
    cond do
      sentinel?(s) ->
        {:done, acc}
      construct?(s) ->
        reduce(%Generator.Character.PrintableASCII{stream: tail(s)}, f.(head(s), acc), f)
    end
  end
end

{:module, Enumerable.Generator.Character.PrintableASCII, <<70, 79, 82, 49, 0, 0, 10, 220, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 1, 166, 0, 0, 0, 39, 52, 69, 108, 105, 120, 105, 114, 46, 69, 110, 117, 109, 101, 114, 97, 98, 108, 101, 46, 71, 101, 110, 101, ...>>, {:__impl__, 1}}

In [68]:
s = Generator.Character.PrintableASCII.stream()
generator = %Generator.Character.PrintableASCII{stream: s}

%Generator.Character.PrintableASCII{stream: [33 | #Function<1.9917893/0 in Caffeine.Stream.map/2>]}

In [69]:
Enum.take(generator, 3)

'!he'

In [70]:
defmodule Generator.Character.PrintableASCII do
  defstruct [
    :stream
  ]

  def stream do
    f = fn (x) ->
      lower_bound() + x
    end
    s = Caffeine.Stream.map(Generator.BoundNatural.stream(limit: length()), f)
    %__MODULE__{
      stream: s
    }
  end

  defp length do
    upper_bound - lower_bound
  end

  defp lower_bound do
    ?\s
  end

  defp upper_bound do
    ?~
  end
end

{:module, Generator.Character.PrintableASCII, <<70, 79, 82, 49, 0, 0, 8, 240, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 1, 77, 0, 0, 0, 29, 41, 69, 108, 105, 120, 105, 114, 46, 71, 101, 110, 101, 114, 97, 116, 111, 114, 46, 67, 104, 97, 114, 97, ...>>, {:upper_bound, 0}}

In [71]:
generator = Generator.Character.PrintableASCII.stream()

%Generator.Character.PrintableASCII{stream: [33 | #Function<1.9917893/0 in Caffeine.Stream.map/2>]}

In [72]:
Enum.take(generator, 3)

'!he'

In [73]:
defimpl Enumerable, for: Generator.Character.PrintableASCII do

  def count(_) do
    {:error, Generator.Character.PrintableASCII}
  end

  def member(_) do
    {:error, Generator.Character.PrintableASCII}
  end

  def count(_) do
    {:error, Generator.Character.PrintableASCII}
  end

  def reduce(s, state, f) do
    _reduce(s.stream, state, f)
  end

  def _reduce(_, {:halt, acc}, _) do
    {:halted, acc}
  end
  def _reduce(s, {:suspend, acc}, f) do
    {:suspend, acc, &_reduce(s, &1, f)}
  end
  def _reduce(s, {:cont, acc}, f) do
    import Caffeine.Stream

    cond do
      sentinel?(s) ->
        {:done, acc}
      construct?(s) ->
        _reduce(tail(s), f.(head(s), acc), f)
    end
  end
end

{:module, Enumerable.Generator.Character.PrintableASCII, <<70, 79, 82, 49, 0, 0, 11, 24, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 1, 164, 0, 0, 0, 39, 52, 69, 108, 105, 120, 105, 114, 46, 69, 110, 117, 109, 101, 114, 97, 98, 108, 101, 46, 71, 101, 110, 101, ...>>, {:__impl__, 1}}

In [74]:
Enum.take(generator, 5)

'!he@E'

In [75]:
3 + 2

5

In [79]:
defmodule S do
  import Caffeine

    def take(s, n) do
      %{ s | stream: _take(s.stream, n)}
    end

    def _take(s, n) when is_integer(n) and n > 0 do
      import Caffeine.Stream, except: [take: 2]

      cond do
        sentinel?(s) ->
          sentinel()

        construct?(s) ->
          g = fn -> _take(tail(s), n - 1) end
          construct(head(s), g)
      end
    end
    def _take(_, 0) do
      Caffeine.Stream.sentinel()
    end
end

{:module, S, <<70, 79, 82, 49, 0, 0, 7, 184, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 1, 10, 0, 0, 0, 30, 8, 69, 108, 105, 120, 105, 114, 46, 83, 8, 95, 95, 105, 110, 102, 111, 95, 95, 9, 102, 117, 110, 99, ...>>, {:_take, 2}}

In [80]:
Enum.take(S.take(generator, 5), 10)

'!he@E'

In [81]:
for e <- S.take(generator, 5) do
  e
end

'!he@E'

In [2]:
defmodule Orange.Natural do
  def stream do
    Stream.iterate(0, &increment/1)
  end

  defp increment(x) do
    x + 1
  end
end

{:module, Orange.Natural, <<70, 79, 82, 49, 0, 0, 5, 32, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 186, 0, 0, 0, 18, 21, 69, 108, 105, 120, 105, 114, 46, 79, 114, 97, 110, 103, 101, 46, 78, 97, 116, 117, 114, 97, 108, 8, ...>>, {:increment, 1}}

In [3]:
Enum.take Orange.Natural.stream(), 5

[0, 1, 2, 3, 4]